In [16]:
import json
from objectbox import (
    Entity,
    Id,
    String,
    Store,
    Box,
    Float32Vector,
    HnswIndex,
    VectorDistanceType,
    
)
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from pathlib import Path
import yaml

In [17]:
rules_chunks_pth = Path("../data/podzemelja_chunks.cleaned.json")
rules_chunks: list[dict] = json.loads(rules_chunks_pth.read_text())
term_ner_pth = Path("../data/terms_merged_ru.json")
term_ner: list[dict] = json.loads(term_ner_pth.read_text())

In [18]:
rules_chunks[0]

{'id': 'main_title_01',
 'content': 'ПОДЗЕМЕЛЬЯ ПЁСИКИ\nПравила игры\n2–4 игрока\nОт 8 лет\n30 минут\nВ волшебном королевстве каждый год проходят соревнования по побегу из подземелья. Самые хорошие и смелые пёсики спускаются в глубокие мрачные комнаты, чтобы сразиться с чудовищами и собрать как можно больше трофеев. Победителей ждут почести и слава, титулы и горы золота, а также очень вкусные косточки.\nВ этой игре вы станете участниками фэнтезийных соревнований. В каждом раунде вы по очереди выкладываете карты комнат, формируя подземные лабиринты с монстрами и сокровищами.\nЗадача каждого игрока — построить наиболее выгодный для себя маршрут к выходу из подземелья, по пути собирая трофеи, монеты и побеждая чудовищ. Победит тот, кто по итогам трёх раундов накопит больше всего золотых монет — их можно вынести из подземелья, получить за выполненные квесты и за побеждённых монстров.',
 'type': 'main_title',
 'section': 'main_title',
 'req_term': ['Пёсики', 'Раунд', 'Победитель'],
 'scenar

In [19]:
term_ner[0]

{'id': 'c3d5b7b9-0dbd-4277-b540-13966774a9c6',
 'name': 'Исследовать подземелье',
 'slug': 'explore-dungeon',
 'kind': 'TERM',
 'path': 'game_mechanics/actions/explore',
 'group': 'game_mechanics',
 'definition': 'Действие, при котором игрок берет верхнюю карту из стопки и выкладывает ее на стол, соблюдая правила построения подземелья.',
 'extra': {'restrictions': ['хотя бы один выход на карте должен совпадать с выходом на другой выложенной карте',
   'карту можно вращать перед выкладыванием']}}

In [20]:
for rule in rules_chunks:
    tags = " ".join(
        [
            "#game:Подземелье_и_Песики",
            f"#section:{rule['section']}",
            f"#type:{rule['type']}",
        ]
    )
    rule["scenario"] = f"{tags}\n---\n{rule["scenario"]}"
    rule["req_term"] = yaml.safe_dump(rule["req_term"], allow_unicode=True)

In [21]:
print(rules_chunks[2]["content"])

Подготовка
Каждый игрок выбирает себе планшет пёсика и кладёт перед собой лицевой стороной вверх. Лишние планшеты уберите в коробку.
Затем каждый игрок берёт себе 7 жетонов лапок, указанных в углу планшета: 5 из них кладёт на планшет (это базовые жетоны), 2 оставшихся кладёт рядом с планшетом (это дополнительные жетоны). Остальные жетоны уберите в коробку.
Перемешайте все карты квестов лицевой стороной вниз и раздайте каждому игроку по 2 карты случайным образом. Лишние карты квестов уберите в коробку.
Каждый игрок выбирает из своих карт квестов одну и кладёт её перед собой лицевой стороной вниз, не показывая остальным. Это личные квесты игроков.
Вторую карту квеста игрок кладёт лицевой стороной вверх на стол, формируя ряд общих квестов. Их может выполнить любой игрок.
Найдите среди карт комнат двустороннюю стартовую комнату. Одна сторона карты предназначена для игры втроём и вчетвером, другая — для игры вдвоём. Положите карту в центре стола нужной стороной вверх — отсюда вы начнёте сво

In [22]:
for term in term_ner:
    tags = " ".join(["#game:Подземелье_и_Песики", f"#group:{term["group"]}"])
    term["content"] = f"{tags}\n---\n{term["name"]}"
    term["extra"] = yaml.safe_dump(term["extra"] if "extra" in term else [], allow_unicode=True)

In [23]:
term_ner[0]

{'id': 'c3d5b7b9-0dbd-4277-b540-13966774a9c6',
 'name': 'Исследовать подземелье',
 'slug': 'explore-dungeon',
 'kind': 'TERM',
 'path': 'game_mechanics/actions/explore',
 'group': 'game_mechanics',
 'definition': 'Действие, при котором игрок берет верхнюю карту из стопки и выкладывает ее на стол, соблюдая правила построения подземелья.',
 'extra': 'restrictions:\n- хотя бы один выход на карте должен совпадать с выходом на другой выложенной карте\n- карту можно вращать перед выкладыванием\n',
 'content': '#game:Подземелье_и_Песики #group:game_mechanics\n---\nИсследовать подземелье'}

In [24]:
from objectbox import Int16


@Entity()
class Rule:
    id = Id
    internal_id = String
    content = String
    section = String
    game= String
    req_term = String
    scenario = String
    priority = Int16
    zone = String
    vector = Float32Vector(
        index=HnswIndex(dimensions=768, distance_type=VectorDistanceType.EUCLIDEAN)
    )
   
@Entity()
class Terminology:
    id = Id
    internal_id = String
    content = String
    name = String
    slug = String
    kind = String
    path = String
    group = String
    definition = String
    extra = String
    vector = Float32Vector(
        index = HnswIndex(dimensions=768)
    )

In [25]:
store = Store(directory="../db")
rules_box = Box(store=store, entity=Rule)
term_ner_box = Box(store, Terminology)

---

In [26]:
model = SentenceTransformer("../model").to("cpu")

In [27]:
box_objects = [
    Rule(
        internal_id=obj["id"],
        content=obj["content"],
        section=obj["section"],
        game="Подземелье и пёсики",
        req_term=obj["req_term"],
        scenario=obj["scenario"],
        priority=obj["priority"],
        zone=obj["zone"],
        vector=model.encode(obj["scenario"]).tolist(),
    )
    for obj in tqdm(rules_chunks)
]

100%|██████████| 19/19 [00:03<00:00,  5.28it/s]


In [28]:
box_terms = [
    Terminology(
        internal_id=obj["id"],
        content=obj["content"],
        name=obj["name"],
        slug=obj["slug"],
        kind=obj["kind"],
        path=obj["path"],
        group=obj["group"],
        definition=obj["definition"],
        extra=obj["extra"],
        vector=model.encode(obj["content"]).tolist(),
    )
    for obj in tqdm(term_ner)
]

100%|██████████| 20/20 [00:02<00:00,  8.38it/s]


In [29]:
rules_box.put(box_objects)
term_ner_box.put(box_terms)

In [30]:
store.close()